In [1]:
#Importing neccessary libraries
import openpyxl 
import pandas as pd
import numpy as np
from openpyxl import load_workbook
data=openpyxl.load_workbook("Excel for Analytics Project Series Source File (1).xlsx")

#Checking the names of sheets in the excel file
for sheets in data.worksheets:
    print(sheets)

<Worksheet "Email">
<Worksheet "EXT0070122021">
<Worksheet "Sheet3">
<Worksheet "Data cleaning">


In [2]:
#Since sheet "EXT0070122021" contains what seems to be volume data, we specifically import sheet "EXT0070122021" as volume
volume=pd.read_excel("Excel for Analytics Project Series Source File (1).xlsx", sheet_name=1)
volume.head(3)

,CLID,Date,Vol
0,CL11420,03/31/2020,884
1,NaN,04/30/2020,886
2,NaN,05/31/2020,968


In [3]:
#check the descriptiion of the data
volume.describe()

,Vol
count,907.000000
mean,5376.309813
std,8551.586979
min,26.000000
25%,568.500000
50%,1173.000000
75%,5284.500000
max,41598.000000


In [4]:
# Check for missing data points
volume.isnull().sum()

CLID    854
Date      0
Vol       0
dtype: int64

In [5]:
# Client id got 854 missing entries. We need to fill it using ffillna function with method ffill to fill values downwards with upper value.sum
volume['CLID']=volume['CLID'].fillna(method='ffill') 
volume.head(4)

,CLID,Date,Vol
0,CL11420,03/31/2020,884
1,CL11420,04/30/2020,886
2,CL11420,05/31/2020,968
3,CL11420,06/30/2020,564


In [6]:
#check the data types of information
volume.dtypes

CLID    object
Date    object
Vol      int64
dtype: object

In [7]:
#We need to change the dtype of date column to dtype
date_format = "%m/%d/%Y"
date_string = volume['Date']
volume['Date']= pd.to_datetime(date_string, format=date_format)

volume['Date'].head()

0   2020-03-31
1   2020-04-30
2   2020-05-31
3   2020-06-30
4   2020-07-31
Name: Date, dtype: datetime64[ns]

In [8]:
# confirm the no of unique id values in volume data
volume_unique_id=len(volume['CLID'].unique())
print(volume_unique_id)

53


In [9]:
# we import GeoData sheet as Geolocation
Geolocation=pd.read_excel("Excel for Analytics Project Series Source File (1).xlsx", sheet_name=2)
Geolocation.head(3)

,CLID,GEOID
0,C-CL69323,GEO1001
1,C-CL97995,GEO1001
2,C-CL87299,GEO1003


In [10]:
#confirm unique id values of Geolocation data
Geolocation_unique=len(Geolocation['CLID'].unique())
print(Geolocation_unique)

53


In [11]:
#We need to trim the number of Geolocation CLID to become 5 characters from 7 so as to be identical to that of volume data
Geolocation['CLID']= Geolocation['CLID'].str[2:]#.str.pad(3, fillchar='0')
Geolocation['CLID'].head()

0    CL69323
1    CL97995
2    CL87299
3    CL38496
4    CL75562
Name: CLID, dtype: object

In [12]:
#Reviewing the email
Email=pd.read_excel("Excel for Analytics Project Series Source File (1).xlsx", sheet_name=0)
print(Email)

Empty DataFrame
Columns: [Hey,

The board is asking to see how volume looked in Q2. I got some data (attached), but didn’t have a chance to pull anything together and was hoping you could take a stab at it.

I think they just want to see Q2 2021 volume by region and wanted to know if everything was looking good. I think this file has what you need. I don’t remember all the region codes – I know NAM ends in 1, EMEA ends in 3 and APAC and LATAM are 2 and 4, but I don’t remember which is which. I do know LATAM has the lowest volume so just go ahead and assign that to which ever comes out lowest.

I appreciate your help!]
Index: []


In [13]:
# We need to merge the dataset into one sheet called GeoVol
GeoVol = volume.merge(Geolocation, on='CLID', how='left')
GeoVol.head()

,CLID,Date,Vol,GEOID
0,CL11420,2020-03-31,884,GEO1004
1,CL11420,2020-04-30,886,GEO1004
2,CL11420,2020-05-31,968,GEO1004
3,CL11420,2020-06-30,564,GEO1004
4,CL11420,2020-07-31,648,GEO1004


In [14]:
# confirming the number of unique client ids to be 53
GeoVol_unique=len(GeoVol['CLID'].unique())
print(GeoVol_unique)

53


In [15]:
#Confirming the unique values of GEOID
print(GeoVol['GEOID'].unique())

['GEO1004' 'GEO1001' 'GEO1003' 'GEO1002']


In [16]:
# We need to inlude Geonames as by description in the email.
#I don’t remember all the region codes – I know NAM ends in 1, EMEA ends in 3 and APAC and LATAM are 2 and 4, but I don’t remember which is which.
# I do know LATAM has the lowest volume so just go ahead and assign that to which ever comes out lowest.

# First we need to calculate the total sales by region.
Sales_volume_by_geoid=GeoVol.groupby('GEOID')['Vol'].sum()
print(Sales_volume_by_geoid)


GEOID
GEO1001    3008286
GEO1002     562005
GEO1003     880760
GEO1004     425262
Name: Vol, dtype: int64


In [17]:
# Since we can clearly see the sales volume by region, we can name each region. 
# we create a mapping dictionary and include a column of region names in our data sheets.
Region_mapping = {"GEO1001": "NAM", "GEO1002":"APAC", "GEO1003":"EMEA","GEO1004":"LATAM"}

GeoVol['REGION_NAME'] = GeoVol['GEOID'].map(Region_mapping)
GeoVol.head()


,CLID,Date,Vol,GEOID,REGION_NAME
0,CL11420,2020-03-31,884,GEO1004,LATAM
1,CL11420,2020-04-30,886,GEO1004,LATAM
2,CL11420,2020-05-31,968,GEO1004,LATAM
3,CL11420,2020-06-30,564,GEO1004,LATAM
4,CL11420,2020-07-31,648,GEO1004,LATAM


In [18]:
# setting index to be Client id 
GeoVol= GeoVol.set_index(GeoVol['CLID'])
GeoVol.head()


,CLID,Date,Vol,GEOID,REGION_NAME
CLID,,,,,
CL11420,CL11420,2020-03-31,884,GEO1004,LATAM
CL11420,CL11420,2020-04-30,886,GEO1004,LATAM
CL11420,CL11420,2020-05-31,968,GEO1004,LATAM
CL11420,CL11420,2020-06-30,564,GEO1004,LATAM
CL11420,CL11420,2020-07-31,648,GEO1004,LATAM


In [19]:

GeoVol.to_csv('GeoVol.csv', index=False)